In [1]:
x=1

In [2]:
import anndata as ad
import multimil as mtm
import muon
import scanpy as sc
import warnings

warnings.filterwarnings("ignore")

[rank: 0] Global seed set to 0


In [9]:
data_path = "/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/trimodal/data/raw/GSE194122_openproblems_neurips2021_cite_BMMC_processed.h5ad"

In [10]:
adata = sc.read_h5ad(data_path)
adata

AnnData object with n_obs × n_vars = 90261 × 14087
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train'
    var: 'feature_types', 'gene_id'
    uns: 'dataset_id', 'genome', 'organism'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [11]:
rna = adata[:, adata.var["feature_types"] == "GEX"].copy()
rna

AnnData object with n_obs × n_vars = 90261 × 13953
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train'
    var: 'feature_types', 'gene_id'
    uns: 'dataset_id', 'genome', 'organism'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [12]:
adt = adata[:, adata.var["feature_types"] == "ADT"].copy()
adt

AnnData object with n_obs × n_vars = 90261 × 134
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train'
    var: 'feature_types', 'gene_id'
    uns: 'dataset_id', 'genome', 'organism'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [13]:
# to free memory
del adata

In [14]:
rna.X = rna.layers["counts"].copy()

In [15]:
sc.pp.normalize_total(rna, target_sum=1e4)
sc.pp.log1p(rna)

In [16]:
n_top_genes = 2000
batch_key = "Site"

In [17]:
sc.pp.highly_variable_genes(rna, n_top_genes=n_top_genes, batch_key=batch_key, subset=True)
rna

AnnData object with n_obs × n_vars = 90261 × 2000
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train'
    var: 'feature_types', 'gene_id', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'dataset_id', 'genome', 'organism', 'log1p', 'hvg'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [18]:
rna.X = rna.layers['counts'].copy()

In [19]:
adt.X = adt.layers["counts"].copy()

In [20]:
muon.prot.pp.clr(adt)
adt.layers["clr"] = adt.X.copy()

In [21]:
rna.obs['split'] = 'train'
rna.obs.loc[rna.obs[batch_key] == "site1", 'split'] = 'test'
rna.obs['split']

GCATTAGCATAAGCGG-1-s1d1     test
TACAGGTGTTAGAGTA-1-s1d1     test
AGGATCTAGGTCTACT-1-s1d1     test
GTAGAAAGTGACACAG-1-s1d1     test
TCCGAAAAGGATCATA-1-s1d1     test
                           ...  
GAATCACCACGGAAGT-1-s4d9    train
GCTGGGTGTACGGATG-1-s4d9    train
TCGAAGTGTGACAGGT-1-s4d9    train
GCAGGCTGTTGCATAC-1-s4d9    train
ACGTAACAGGTCTACT-1-s4d9    train
Name: split, Length: 90261, dtype: object

In [22]:
adt.obs = adt.obs.join(rna.obs['split'])
adt.obs['split']

GCATTAGCATAAGCGG-1-s1d1     test
TACAGGTGTTAGAGTA-1-s1d1     test
AGGATCTAGGTCTACT-1-s1d1     test
GTAGAAAGTGACACAG-1-s1d1     test
TCCGAAAAGGATCATA-1-s1d1     test
                           ...  
GAATCACCACGGAAGT-1-s4d9    train
GCTGGGTGTACGGATG-1-s4d9    train
TCGAAGTGTGACAGGT-1-s4d9    train
GCAGGCTGTTGCATAC-1-s4d9    train
ACGTAACAGGTCTACT-1-s4d9    train
Name: split, Length: 90261, dtype: object

In [23]:
del rna.layers
del adt.layers
del rna.var
del adt.var
del rna.obsm
del adt.obsm

In [24]:
rna.write('../pipeline/data/cite_rna.h5ad')
adt.write('../pipeline/data/cite_adt.h5ad')

In [25]:
rna.X.data

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [26]:
adt.X.data

array([0.38831878, 3.2994409 , 0.38831878, ..., 1.1925837 , 0.9706666 ,
       0.7386633 ], dtype=float32)